# Extracting reviews

In this comments there are commercial items mentioned. Where this type of entities are found, the idea is to match the part of the text that's referring to each particular item. 




In [1]:
import pickle
import boto3
import re
import logging

In [4]:
reviews = []

In [5]:
comprehend = boto3.client('comprehend')

In [6]:
def return_targets(entities):
    #TO DO Detect Organizations and Brands
    comm_items = []
    for entity in entities:
        if entity['Score'] > 0.6 and entity['Type'] == 'COMMERCIAL_ITEM':
            comm_items.append(entity)
            print(entity)
    #Eliminate repeated entities
    existing_items = []
    for item in comm_items[:]:
        if item['Text'] in existing_items:
            comm_items.remove(item)
        else:
            existing_items.append(item['Text'])
            
    return comm_items

In [7]:
def targets_are_all_equal(targets):
    check_term = targets[0]['Text'].lower() 
    for i in range(1,len(target)):
        if targets[i]['Text'].lower() != check_term:
            return False
    return True

In [8]:
def get_phrases_index_comment(comment):
    phrase_sep = [regx.end() for regx in list(re.finditer('\.',comment))]
    phrase_sep.insert(0,0)
    return phrase_sep

In [9]:
def process_comment(comment):
    comment_reviews = []
    entities = comprehend.detect_entities(Text=comment,LanguageCode='en')['Entities']
    commercial_items = [ent for ent in entities if ent['Type'] == 'COMMERCIAL_ITEM']
    if len(commercial_items) == 0:
        logging.info('No commercial item found')
    if len(commercial_items) == 1:
        comment_reviews.append({'review_text':comment,'item':commercial_items[0]['Text']})
    if len(commercial_items) > 1:
        comment_reviews = find_reviews_sentences(comment,commercial_items)
    
    return comment_reviews

In [10]:
def get_sentences_and_items(comment,commercial_items):
    phrases_index = get_phrases_index_comment(comment)
    sentences = []
    for i in range(len(phrases_index) -1):
        entities_in_sentence = [item for item in commercial_items if item['BeginOffset'] >= phrases_index[i] 
                                and  item['EndOffset'] <= phrases_index[i + 1] ]
        phrase = comment[int(phrases_index[i]):int(phrases_index[i+1])] 
        sentences.append({'phrase':phrase,'entities':entities_in_sentence}) 
    return sentences

In [11]:
def find_reviews_sentences(comment,commercial_items):
    
    reviews = []
    sentences = get_sentences_and_items(comment,commercial_items)
    review = ''
    for sentence in sentences:
        review += sentence['phrase']
        for entity in sentence['entities']:
            reviews.append({'review_text':review,'item':entity['Text']})
            
        if len(sentence['entities']) > 0:
            #Prepare next review
            review = ''
    return reviews


In [2]:
with open('submissions/best_&_worst_of:_liquid_foundations.pickle', 'rb') as handle:
    comments = pickle.load(handle)


Processing each comment to find the review and the item it refers to. 
When there's only one entity in the comment, I'll assume the full text referers to that entity.
When there is more than one, 


In [12]:
for i in range(0,len(comments)):
    print ('#' * 30)
    comment = comments[i]['text']
    print(comment)
    print ('*' * 30)
    
    revs = process_comment(comment)
    
    print (f'Found {len(revs)} reviews')
    print ('-' * 30)
    print(revs)
    print ('-' * 30)

##############################
Love: Tarte Amazonian Clay BB Tinted Moisturizer Broad Spectrum SPF 20 Sunscreen.I really like this product because it provides light coverage, has spf protection, and contains vitamins. The coverage is enough to conceal redness on my face and it lasts all day. I don't find it to feel heavy or sticky. .In the winter, when it is super dry/cold/windy where I live, I apply a gel moisturizer 5 min. before putting this on.I find the colour 'light' works for me. I have normal skin type and I am NC 15/20 (I haven't bought from MAC in a long time so that might not be accurate).
******************************
Found 1 reviews
------------------------------
[{'review_text': "Love: Tarte Amazonian Clay BB Tinted Moisturizer Broad Spectrum SPF 20 Sunscreen.I really like this product because it provides light coverage, has spf protection, and contains vitamins. The coverage is enough to conceal redness on my face and it lasts all day. I don't find it to feel heavy or s

In [13]:
with open('submissions/best_and_worst_iphone.pickle', 'rb') as handle:
    comments = pickle.load(handle)


In [14]:
for i in range(0,len(comments)):
    print ('#' * 30)
    comment = comments[i]['text']
    print(comment)
    print ('*' * 30)
    
    revs = process_comment(comment)
    
    print (f'Found {len(revs)} reviews')
    print ('-' * 30)
    print(revs)
    print ('-' * 30)

##############################
iPhone 6s.The leap from 6 to 6s was astronomical even to this day.
******************************
Found 2 reviews
------------------------------
[{'review_text': 'iPhone 6s.The leap from 6 to 6s was astronomical even to this day.', 'item': '6'}, {'review_text': 'iPhone 6s.The leap from 6 to 6s was astronomical even to this day.', 'item': '6s'}]
------------------------------
##############################
True,the 6 was not good enough,from bendgate issue and the 1gb ram is also the case why the 6 dint get ios 13 and above.
******************************
Found 2 reviews
------------------------------
[{'review_text': 'True,the 6 was not good enough,from bendgate issue and the 1gb ram is also the case why the 6 dint get ios 13 and above.', 'item': '6'}, {'review_text': 'True,the 6 was not good enough,from bendgate issue and the 1gb ram is also the case why the 6 dint get ios 13 and above.', 'item': '6'}]
------------------------------
#####################